# Compiling 2013> New York School data into similar format to latest years.

After downloading the [2012 data from the New York State Education Department](https://data.nysed.gov/downloads.php) and exporting the .mdb file into csv(s), this program gets these years into the same format as the latest years allowing for historical comparisons.

In [7]:
import agate
import csv
import agateexcel

Due to missing school values and changes in data columns, I needed two different typetesters to force the columns into their correct formatting.

In [8]:
tester_15 = agate.TypeTester(limit=100,force={
    'NRC_DESC': agate.Text(),
    'NRC_CODE': agate.Number(),
        'BEDSCODE': agate.Text(),
        'L1_COUNT': agate.Text(),
        'L1_PCT': agate.Text(),
        'L2_COUNT': agate.Text(),
        'L2_PCT': agate.Text(),
        'L3_COUNT': agate.Text(),
        'L3_PCT': agate.Text(),
        'L4_COUNT': agate.Text(),
        'L4_PCT': agate.Text(),
        'L2-L4_PCT': agate.Text(),
        'L3-L4_PCT': agate.Text(),
        'MEAN_SCALE_SCORE': agate.Text(),
        'COUNTY_DESC': agate.Text(),
        'TOTAL_TESTED': agate.Text(),
        'COUNTY_CODE': agate.Text(),
        'Sum_Of_SCALE_SCORE': agate.Text()
})
tester_16 = agate.TypeTester(limit=100,force={
    'NRC_DESC': agate.Text(),
    'NRC_CODE': agate.Number(),
        'BEDSCODE': agate.Text(),
        'L1_COUNT': agate.Text(),
        'L1_PCT': agate.Text(),
        'L2_COUNT': agate.Text(),
        'L2_PCT': agate.Text(),
        'L3_COUNT': agate.Text(),
        'L3_PCT': agate.Text(),
        'L4_COUNT': agate.Text(),
        'L4_PCT': agate.Text(),
        'L2-L4_PCT': agate.Text(),
        'L3-L4_PCT': agate.Text(),
        'MEAN_SCALE_SCORE': agate.Text(),
        'COUNTY_DESC': agate.Text(),
        'TOTAL_TESTED': agate.Text(),
        'COUNTY_CODE': agate.Text()
})

This program takes in the csv of a specific year's assesment data and its tester and returns a dictionary of BEDS (state id codes) and test data.

In [9]:
comparison_dict = {}

with open('raw/3-8_ELA_AND_MATH_2017.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['BEDSCODE'] not in comparison_dict and row['SUBGROUP_NAME'] == 'All Students':
            comparison_dict[row['BEDSCODE']] = row

In [10]:
print(comparison_dict['400000000000'])

OrderedDict([('SY_END_DATE', '06/30/2017'), ('NRC_CODE', ''), ('NRC_DESC', ''), ('COUNTY_CODE', '40'), ('COUNTY_DESC', 'NIAGARA'), ('BEDSCODE', '400000000000'), ('NAME', 'NIAGARA COUNTY'), ('ITEM_SUBJECT_AREA', 'ELA'), ('ITEM_DESC', 'Grade 3 ELA'), ('SUBGROUP_CODE', '01'), ('SUBGROUP_NAME', 'All Students'), ('TOTAL_TESTED', '1722'), ('L1_COUNT', '495'), ('L1_PCT', '29%'), ('L2_COUNT', '558'), ('L2_PCT', '32%'), ('L3_COUNT', '551'), ('L3_PCT', '32%'), ('L4_COUNT', '118'), ('L4_PCT', '7%'), ('L2-L4_PCT', '71%'), ('L3-L4_PCT', '39%'), ('MEAN_SCALE_SCORE', '306')])


In [6]:
unmatched = {}

def mani_table(table, subgroup, year_folder, clean_info0, clean_info1):
    year_key0 = year_folder[13:15]
    year_key1 = year_folder[15:17]
    for row in table.rows:
        if row['YEAR'] == '2012' and year_folder == 'school_scores2012':
            clean_dict = {}
            geoid = row['ENTITY_CD']
            if 'County' in row[1]:
                #County codes have 0001400000 where current use 140000000
                geoid = row['ENTITY_CD'][4:] + row['ENTITY_CD'][0:4]
            if geoid not in comparison_dict:
                unmatched[geoid] = row['ENTITY_NAME']
            else:
                subject = subgroup[:-1]
                grade = subgroup[-1]
                current_info = comparison_dict[geoid]
                clean_dict['COUNTY_DESC'] = current_info['COUNTY_DESC']
                clean_dict['COUNTY_CODE'] = current_info['COUNTY_CODE']
                clean_dict['SUBGROUP_NAME'] = row['SUBGROUP_NAME']
                clean_dict['BEDSCODE'] = geoid
                clean_dict['NAME'] = current_info['NAME']
                clean_dict['TOTAL_TESTED'] = row['NUM_TESTED']
                clean_dict['L1_COUNT'] = row['LEVEL1_COUNT']
                clean_dict['L2_COUNT'] = row['LEVEL2_COUNT']
                clean_dict['L3_COUNT'] = row['LEVEL3_COUNT']
                clean_dict['L4_COUNT'] = row['LEVEL4_COUNT']
                clean_dict['ITEM_DESC'] = "Grade {0} {1}".format(grade, subject)
                if subject == 'Math':
                    clean_dict['ITEM_SUBJECT_AREA'] = 'Mathematics'
                else:
                    clean_dict['ITEM_SUBJECT_AREA'] = subject
                clean_dict['MEAN_SCALE_SCORE'] = row['MEAN_SCORE']
                clean_info0.append(clean_dict)
        elif year_folder != 'school_scores2012':
            clean_dict = {}
            geoid = row['ENTITY_CD']
            if 'County' in row[1]:
                #County codes have 0001400000 where current use 140000000
                geoid = row['ENTITY_CD'][4:] + row['ENTITY_CD'][0:4]
            if geoid not in comparison_dict:
                unmatched[geoid] = row['ENTITY_NAME']
            else:
                subject = subgroup[:-1]
                grade = subgroup[-1]
                current_info = comparison_dict[geoid]
                clean_dict['COUNTY_DESC'] = current_info['COUNTY_DESC']
                clean_dict['COUNTY_CODE'] = current_info['COUNTY_CODE']
                clean_dict['SUBGROUP_NAME'] = row['SUBGROUP_NAME']
                clean_dict['BEDSCODE'] = geoid
                clean_dict['NAME'] = current_info['NAME']
                clean_dict['TOTAL_TESTED'] = row['NUM_TESTED']
                clean_dict['L1_COUNT'] = row['LEVEL1_COUNT']
                clean_dict['L2_COUNT'] = row['LEVEL2_COUNT']
                clean_dict['L3_COUNT'] = row['LEVEL3_COUNT']
                clean_dict['L4_COUNT'] = row['LEVEL4_COUNT']
                clean_dict['ITEM_DESC'] = "Grade {0} {1}".format(grade, subject)
                if subject == 'Math':
                    clean_dict['ITEM_SUBJECT_AREA'] = 'Mathematics'
                else:
                    clean_dict['ITEM_SUBJECT_AREA'] = subject
                clean_dict['MEAN_SCALE_SCORE'] = row['MEAN_SCORE']
                if row['YEAR'][2:] == year_key0:
                    clean_info0.append(clean_dict)
                else:
                    clean_info1.append(clean_dict)

results = ['ELA3 Subgroup Results', 'ELA4 Subgroup Results','ELA5 Subgroup Results','ELA6 Subgroup Results','ELA7 Subgroup Results', 'ELA8 Subgroup Results', 'Math3 Subgroup Results', 'Math4 Subgroup Results', 'Math5 Subgroup Results', 'Math6 Subgroup Results', 'Math7 Subgroup Results', 'Math8 Subgroup Results']
def grab_subgroup(csvpath, year_folder, clean_info0, clean_info1):
    subgroup = csvpath.split(' ')
    full_path = 'raw/' + year_folder +'/'+ csvpath + '.xlsx'
    tester = agate.TypeTester(limit=100,force={
    'ENTITY_CD': agate.Text(),
    'NUM_TESTED': agate.Text(),
    'LEVEL1_COUNT': agate.Text(),
    'LEVEL2_COUNT': agate.Text(),
    'LEVEL3_COUNT': agate.Text(),
    'LEVEL4_COUNT': agate.Text(),
    'NUM_TESTED': agate.Text(),
    'LEVEL1_%TESTED': agate.Text(),
    'LEVEL2_%TESTED': agate.Text(),
    'LEVEL3_%TESTED': agate.Text(),
    'LEVEL4_%TESTED': agate.Text(),
    'TOTAL_SCALE_SCORES': agate.Text(),
    'MEAN_SCORE': agate.Text(),
    'YEAR': agate.Text()
    })
    schools = agate.Table.from_xlsx(full_path, column_types=tester)
    print(len(schools))
    print('clean_info0={0};clean_info1={1}'.format(len(clean_info0), len(clean_info1)))
    mani_table(schools, subgroup[0], year_folder, clean_info0, clean_info1)

folders = ['school_scores1011', 'school_scores0809', 'school_scores0607', 'school_scores2012']

for folder in folders:
    clean_info0 = []
    clean_info1 = []
    print('Accessing {0}'.format(folder))
    for result in results:
        print('Grabbing {0}'.format(result))
        grab_subgroup(result, folder, clean_info0, clean_info1)
    if folder == 'school_scores2012':
        csv_path = 'raw/3-8_ELA_AND_MATH_2012.csv'
        with open(csv_path, 'w') as csvfile:
            fieldnames = clean_info0[0].keys()
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for row in clean_info:
                writer.writerow(row)
    else:
        csv_path0 = 'raw/3-8_ELA_AND_MATH_20{}.csv'.format(folder[13:15])
        print(csv_path0)
        csv_path1 = 'raw/3-8_ELA_AND_MATH_20{}.csv'.format(folder[15:17])
        print(csv_path1)
        with open(csv_path0, 'w') as csvfile:
            fieldnames = clean_info0[0].keys()
            print(fieldnames)
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for row in clean_info0:
                writer.writerow(row)
        with open(csv_path1, 'w') as csvfile:
            fieldnames = clean_info1[0].keys()
            print(fieldnames)
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for row in clean_info1:
                writer.writerow(row)         

    

Accessing school_scores1011
Grabbing ELA3 Subgroup Results


FileNotFoundError: [Errno 2] No such file or directory: 'raw/school_scores1011/ELA3 Subgroup Results.xlsx'

In [170]:
print(clean_info[1359])
print(comparison_dict['490000000000'])

{'COUNTY_DESC': 'RENSSELAER', 'ITEM_DESC': 'Grade 3 ELA', 'L4_COUNT': '2', 'ITEM_SUBJECT_AREA': 'ELA', 'L2_COUNT': '71', 'BEDSCODE': '490000000000', 'SUBGROUP_NAME': 'Students with Disabilities', 'NAME': 'RENSSELAER COUNTY', 'COUNTY_CODE': '49', 'MEAN_SCALE_SCORE': '643', 'TOTAL_TESTED': '216', 'L1_COUNT': '104', 'L3_COUNT': '39'}
{'COUNTY_DESC': 'RENSSELAER', 'L3-L4_PCT': '44%', 'L2_COUNT': '421', 'L1_PCT': '25%', 'L4_PCT': '8%', 'COUNTY_CODE': '49', 'SUBGROUP_CODE': '01', 'L3_COUNT': '493', 'SUBGROUP_NAME': 'All Students', 'NAME': 'RENSSELAER COUNTY', 'TOTAL_TESTED': '1374', 'L1_COUNT': '349', 'L4_COUNT': '111', 'ITEM_DESC': 'Grade 3 ELA', 'L2-L4_PCT': '75%', 'NRC_CODE': '', 'NRC_DESC': '', 'SY_END_DATE': '06/30/2017', 'BEDSCODE': '490000000000', 'ITEM_SUBJECT_AREA': 'ELA', 'MEAN_SCALE_SCORE': '309', 'L2_PCT': '31%', 'L3_PCT': '36%'}


In [171]:
print(len(unmatched.keys()))
print(unmatched)

619
{'280210030006': 'MILBURN ELEMENTARY SCHOOL', '660412020003': 'CLARK ACADEMY', '321000011237': 'MARIE CURIE HIGH SCH-NURSING, MEDICINE & APPLIED HLTH PROF', '321100011270': 'ACADEMY FOR SCHOLARSHIP AND ENTRENEURSHIP', '571502060003': 'CANISTEO-GREENWOOD MIDDLE SCHOOL', '140600010103': 'GROVER CLEVELAND HIGH SCHOOL', '401301040004': 'BARKER MIDDLE SCHOOL', '310200860905': 'ROSS GLOBAL ACADEMY CHARTER SCHOOL', '001800000000': 'Similar Schools Group #18', '491302060003': 'GEORGE WASHINGTON SCHOOL', '651201060002': 'SODUS INTERMEDIATE SCHOOL', '007400000000': 'Similar Schools Group #74', '660401030002': 'W L MORSE SCHOOL', '180300010004': 'ROBERT MORRIS SCHOOL', '251601060005': 'CHITTENANGO HIGH SCHOOL', '460801060004': 'CLEVELAND ELEMENTARY SCHOOL', '321200011270': 'ACADEMY FOR SCHOLARSHIP AND ENTRENEURSHIP', '331700860951': 'FAHARI ACADEMY CHARTER SCHOOL', '091101060006': 'PERU MIDDLE SCHOOL', '331300010103': 'SATELLITE THREE', '261600010036': 'SCHOOL 36-HENRY W LONGFELLOW', '5306000